## Sudoku Solver

A brute force solver class with generators.

In [1]:
import numpy as np
import unittest
import math 

class Sudoku(object):
    """
    A simple brute force solver for sudoku puzzles.
    
    Parameters
    -----------
        board: numpy.ndarray of shape (9,9)
    
    Methods
    -----------
        solve() 
            returns <generator object Sudoku.solve ... >
            usage example,
                solution_generator = Sudoku(board).solve()
                next(solution_generator).view()
    """
    
    MAXROW = 9
    MAXCOL = 9
    
    def __init__(self, board=np.zeros((9, 9), dtype='int64')):
        if not isinstance(board, np.ndarray) or board.shape != (9,9):
            raise ValueError('Only supports a numpy.ndarray of shape (9,9) at current.')
        self.board = board

    def solve(self):
        """
        Create a generartor of boards that may be lazily evaluated.
        """
        
        for row in range (0, Sudoku.MAXROW):
            for col in range (0, Sudoku.MAXCOL):
                if self.board[row][col] == 0:
                    for num in range (1, Sudoku.MAXROW+1):
                        if self._is_valid_placement(row, col, num):
                            self.board[row][col] = num
                            yield from self.solve()
                            self.board[row][col] = 0
                    return
        yield self.board
            
    def _is_valid_placement(self, row, col, num):
        """
        Returns True if we can place num at self.board[row][col] and not violate any rules.
        """
        
        if not (0 <= row < Sudoku.MAXROW) or not (0 <= col < Sudoku.MAXCOL):
            raise ValueError(f'({row}, {col}) is off the board; the maximum is ({Sudoku.MAXROW}, {Sudoku.MAXCOL}).')
                
        if (self._is_valid_placement_row(row, col, num) and 
            self._is_valid_placement_col(row, col, num) and 
            self._is_valid_placement_square(row, col, num)):
            return True
        
        return False
    
    def _is_valid_placement_row(self, row, col, num):
        for i in range(0, Sudoku.MAXROW):
            if self.board[row][i] == num:
                return False
        return True
    
    def _is_valid_placement_col(self, row, col, num):
        for i in range(0, Sudoku.MAXCOL):
            if self.board[i][col] == num:
                return False
        return True
    
    def _is_valid_placement_square(self, row, col, num):
        square_len = math.floor(math.sqrt(Sudoku.MAXROW))
        for i in range(math.floor(row / square_len) * square_len, math.floor(row / square_len) * square_len + square_len):
            for j in range(math.floor(col / square_len) * square_len, math.floor(col / square_len) * square_len + square_len):
                if self.board[i][j] == num:
                    return False
        return True


#### Solve a real puzzle

In [2]:
expert = np.array([[0, 0, 0, 0, 5, 0, 8, 0, 0],
                   [0, 0, 7, 0, 0, 1, 6, 0, 0],
                   [0, 0, 0, 7, 0, 2, 0, 3, 0],
                   [0, 0, 8, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 2, 9, 0, 3, 7, 0],          
                   [4, 0, 0, 0, 0, 0, 0, 0, 5],
                   [8, 0, 2, 0, 0, 7, 0, 0, 0],
                   [0, 4, 0, 0, 3, 0, 0, 1, 8],
                   [0, 0, 6, 0, 0, 0, 0, 0, 0]])

expert_puzzle = Sudoku(expert)
solution = expert_puzzle.solve()  # solution is a generator of solutions
next(solution).view()

array([[6, 1, 3, 4, 5, 9, 8, 2, 7],
       [2, 5, 7, 3, 8, 1, 6, 4, 9],
       [9, 8, 4, 7, 6, 2, 5, 3, 1],
       [3, 7, 8, 1, 4, 5, 9, 6, 2],
       [5, 6, 1, 2, 9, 8, 3, 7, 4],
       [4, 2, 9, 6, 7, 3, 1, 8, 5],
       [8, 3, 2, 5, 1, 7, 4, 9, 6],
       [7, 4, 5, 9, 3, 6, 2, 1, 8],
       [1, 9, 6, 8, 2, 4, 7, 5, 3]])

#### Blank boards have solutions too.

_many_ in fact, so this is where the generator is useful.

In [3]:
# create board, load solution set into a generator
board = np.zeros((9, 9), dtype='int64')
puzzle = Sudoku(board)
solution = puzzle.solve()  # solution is a generator of solutions

In [4]:
# first solution 
next(solution).view()

array([[1, 2, 3, 4, 5, 6, 7, 8, 9],
       [4, 5, 6, 7, 8, 9, 1, 2, 3],
       [7, 8, 9, 1, 2, 3, 4, 5, 6],
       [2, 1, 4, 3, 6, 5, 8, 9, 7],
       [3, 6, 5, 8, 9, 7, 2, 1, 4],
       [8, 9, 7, 2, 1, 4, 3, 6, 5],
       [5, 3, 1, 6, 4, 2, 9, 7, 8],
       [6, 4, 2, 9, 7, 8, 5, 3, 1],
       [9, 7, 8, 5, 3, 1, 6, 4, 2]])

In [5]:
# second solution
next(solution).view()

array([[1, 2, 3, 4, 5, 6, 7, 8, 9],
       [4, 5, 6, 7, 8, 9, 1, 2, 3],
       [7, 8, 9, 1, 2, 3, 4, 5, 6],
       [2, 1, 4, 3, 6, 5, 8, 9, 7],
       [3, 6, 5, 8, 9, 7, 2, 1, 4],
       [8, 9, 7, 2, 1, 4, 3, 6, 5],
       [5, 3, 1, 6, 4, 2, 9, 7, 8],
       [6, 4, 8, 9, 7, 1, 5, 3, 2],
       [9, 7, 2, 5, 3, 8, 6, 4, 1]])

_... we could do this for a long time._

#### Tests

In [6]:
class TestSudoku(unittest.TestCase):
    
    def test_is_valid_placement(self):
        board = np.zeros((9, 9), dtype='int64')
        board[1][4] = 1
        board[8][7] = 1
        puzzle = Sudoku(board)
        
        self.assertFalse(puzzle._is_valid_placement(1,3,1)) # row test
        self.assertFalse(puzzle._is_valid_placement(0,4,1)) # col test
        self.assertFalse(puzzle._is_valid_placement(8,8,1)) # square test
        self.assertTrue(puzzle._is_valid_placement(2,3,2))
        
         # Test boundaries
        self.assertRaises(ValueError, puzzle._is_valid_placement, 2, 9, 1)
    
    def test_solution(self):
        # board with only one outcome
        board = np.array([[0, 0, 0, 6, 8, 0, 0, 0, 0],
                          [0, 6, 0, 0, 0, 0, 5, 0, 0],
                          [5, 0, 0, 0, 3, 0, 0, 0, 0],
                          [0, 0, 0, 9, 2, 0, 0, 0, 0],
                          [0, 0, 6, 0, 0, 0, 0, 0, 1],          
                          [3, 4, 0, 0, 0, 0, 0, 0, 6],
                          [0, 1, 0, 3, 0, 0, 0, 0, 7],
                          [0, 0, 7, 0, 0, 0, 9, 0, 0],
                          [0, 8, 0, 0, 9, 0, 2, 5, 0]])
        # desired outcome
        sol = np.array([[1, 3, 2, 6, 8, 5, 4, 7, 9], 
                        [7, 6, 4, 2, 1, 9, 5, 3, 8],
                        [5, 9, 8, 7, 3, 4, 1, 6, 2],
                        [8, 7, 1, 9, 2, 6, 3, 4, 5],
                        [2, 5, 6, 4, 7, 3, 8, 9, 1],
                        [3, 4, 9, 8, 5, 1, 7, 2, 6],
                        [9, 1, 5, 3, 4, 2, 6, 8, 7],
                        [4, 2, 7, 5, 6, 8, 9, 1, 3],
                        [6, 8, 3, 1, 9, 7, 2, 5, 4]])
        # compute and see.
        expert_puzzle = Sudoku(board)
        solution = expert_puzzle.solve()
        derived_solution = next(solution)
        
        # test
        self.assertTrue((derived_solution == sol).all())

unittest.main(argv=[''], verbosity=2, exit=False)

test_is_valid_placement (__main__.TestSudoku) ... ok
test_solution (__main__.TestSudoku) ... ok

----------------------------------------------------------------------
Ran 2 tests in 31.389s

OK
